<a href="https://colab.research.google.com/github/Wilma1088/mypackage/blob/master/Zindi_Challenge_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Honour Code**
I Wilma Nel, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

## Downloaded data
Downloaded data files from: https://zindi.africa/hackathons/edsa-sendy-logistics-challenge/data

## Imports

Imported files to utilise throughout notebook.

Additional imports may be noted within the notebook.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from tqdm import tqdm
import math
import random

from scipy.stats import norm, skew #for some statistics

from google.colab import files
import io

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Explanatory Data Analysis (EDA)
For this predit I utilised the datasets provided on the Sendy platform.

In [ ]:
uploaded = files.upload()

Saving Riders.csv to Riders.csv
Saving SampleSubmission.csv to SampleSubmission.csv
Saving Test.csv to Test.csv
Saving Train.csv to Train.csv
Saving VariableDefinitions.csv to VariableDefinitions.csv


**Riders Dataframe**

In [ ]:
riders_df = pd.read_csv(io.BytesIO(uploaded['Riders.csv']))
riders_df.columns= [col.replace(' ', '_') for col in riders_df.columns]
riders_df.head()

,Rider_Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176
2,Rider_Id_648,1746,821,14.3,466
3,Rider_Id_753,314,980,12.5,75
4,Rider_Id_335,536,1113,13.7,156


**Test Dataframe**

In [ ]:
test_df = pd.read_csv(io.BytesIO(uploaded['Test.csv']))
test_df.columns= [col.replace(' ', '_') for col in test_df.columns]
test_df.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Day_of_Month,Confirmation_-_Weekday_(Mo_=_1),Confirmation_-_Time,Arrival_at_Pickup_-_Day_of_Month,Arrival_at_Pickup_-_Weekday_(Mo_=_1),Arrival_at_Pickup_-_Time,Pickup_-_Day_of_Month,Pickup_-_Weekday_(Mo_=_1),Pickup_-_Time,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Rider_Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,4:44:29 PM,27,3,4:53:04 PM,27,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,12:59:17 PM,17,5,1:20:27 PM,17,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,11:25:05 AM,27,4,11:33:20 AM,27,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,1:53:27 PM,17,1,2:02:41 PM,17,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,11:34:45 AM,11,2,11:47:19 AM,11,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


**Train Dataframe**

In [ ]:
train_df = pd.read_csv(io.BytesIO(uploaded['Train.csv']))
train_df.columns= [col.replace(' ', '_') for col in train_df.columns]
train_df.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Day_of_Month,Confirmation_-_Weekday_(Mo_=_1),Confirmation_-_Time,Arrival_at_Pickup_-_Day_of_Month,Arrival_at_Pickup_-_Weekday_(Mo_=_1),Arrival_at_Pickup_-_Time,Pickup_-_Day_of_Month,Pickup_-_Weekday_(Mo_=_1),Pickup_-_Time,Arrival_at_Destination_-_Day_of_Month,Arrival_at_Destination_-_Weekday_(Mo_=_1),Arrival_at_Destination_-_Time,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Rider_Id,Time_from_Pickup_to_Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


**Merging Datasets**

In [ ]:
riders_df.columns

Index(['Rider_Id', 'No_Of_Orders', 'Age', 'Average_Rating', 'No_of_Ratings'], dtype='object')

In [ ]:
train_df.columns

Index(['Order_No', 'User_Id', 'Vehicle_Type', 'Platform_Type',
       'Personal_or_Business', 'Placement_-_Day_of_Month',
       'Placement_-_Weekday_(Mo_=_1)', 'Placement_-_Time',
       'Confirmation_-_Day_of_Month', 'Confirmation_-_Weekday_(Mo_=_1)',
       'Confirmation_-_Time', 'Arrival_at_Pickup_-_Day_of_Month',
       'Arrival_at_Pickup_-_Weekday_(Mo_=_1)', 'Arrival_at_Pickup_-_Time',
       'Pickup_-_Day_of_Month', 'Pickup_-_Weekday_(Mo_=_1)', 'Pickup_-_Time',
       'Arrival_at_Destination_-_Day_of_Month',
       'Arrival_at_Destination_-_Weekday_(Mo_=_1)',
       'Arrival_at_Destination_-_Time', 'Distance_(KM)', 'Temperature',
       'Precipitation_in_millimeters', 'Pickup_Lat', 'Pickup_Long',
       'Destination_Lat', 'Destination_Long', 'Rider_Id',
       'Time_from_Pickup_to_Arrival'],
      dtype='object')

In [39]:
test_df.columns

Index(['Order_No', 'User_Id', 'Vehicle_Type', 'Platform_Type',
       'Personal_or_Business', 'Placement_-_Day_of_Month',
       'Placement_-_Weekday_(Mo_=_1)', 'Placement_-_Time',
       'Confirmation_-_Day_of_Month', 'Confirmation_-_Weekday_(Mo_=_1)',
       'Confirmation_-_Time', 'Arrival_at_Pickup_-_Day_of_Month',
       'Arrival_at_Pickup_-_Weekday_(Mo_=_1)', 'Arrival_at_Pickup_-_Time',
       'Pickup_-_Day_of_Month', 'Pickup_-_Weekday_(Mo_=_1)', 'Pickup_-_Time',
       'Distance_(KM)', 'Temperature', 'Precipitation_in_millimeters',
       'Pickup_Lat', 'Pickup_Long', 'Destination_Lat', 'Destination_Long',
       'Rider_Id'],
      dtype='object')

In [ ]:
#train_df = train_df[['Order_No', 'User_Id', 'Vehicle_Type', 'Platform_Type',
#       'Personal_or_Business', 'Placement_-_Day_of_Month',
#       'Placement_-_Weekday_(Mo_=_1)', 'Placement_-_Time',
#       'Confirmation_-_Day_of_Month', 'Confirmation_-_Weekday_(Mo_=_1)',
#       'Confirmation_-_Time', 'Arrival_at_Pickup_-_Day_of_Month',
#       'Arrival_at_Pickup_-_Weekday_(Mo_=_1)', 'Arrival_at_Pickup_-_Time',
#       'Pickup_-_Day_of_Month', 'Pickup_-_Weekday_(Mo_=_1)', 'Pickup_-_Time',
#       'Distance_(KM)', 'Temperature', 'Precipitation_in_millimeters',
#       'Pickup_Lat', 'Pickup_Long', 'Destination_Lat', 'Destination_Long',
#       'Rider_Id', 'Time_from_Pickup_to_Arrival']]

In [42]:
df1 = pd.concat([test_df, train_df], axis=0, ignore_index=True)
df1.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Day_of_Month,Confirmation_-_Weekday_(Mo_=_1),Confirmation_-_Time,Arrival_at_Pickup_-_Day_of_Month,Arrival_at_Pickup_-_Weekday_(Mo_=_1),Arrival_at_Pickup_-_Time,Pickup_-_Day_of_Month,Pickup_-_Weekday_(Mo_=_1),Pickup_-_Time,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Rider_Id,Time_from_Pickup_to_Arrival
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,4:44:29 PM,27,3,4:53:04 PM,27,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192,NaN
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,12:59:17 PM,17,5,1:20:27 PM,17,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868,NaN
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,11:25:05 AM,27,4,11:33:20 AM,27,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26,NaN
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,1:53:27 PM,17,1,2:02:41 PM,17,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685,NaN
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,11:34:45 AM,11,2,11:47:19 AM,11,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858,NaN


In [ ]:
#df=pd.concat([train_df, test_df])
#df.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Day_of_Month,Confirmation_-_Weekday_(Mo_=_1),Confirmation_-_Time,Arrival_at_Pickup_-_Day_of_Month,Arrival_at_Pickup_-_Weekday_(Mo_=_1),Arrival_at_Pickup_-_Time,Pickup_-_Day_of_Month,Pickup_-_Weekday_(Mo_=_1),Pickup_-_Time,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Rider_Id,Time_from_Pickup_to_Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745.0
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993.0
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455.0
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341.0
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214.0


In [43]:
df1.dtypes

Order_No                                 object
User_Id                                  object
Vehicle_Type                             object
Platform_Type                             int64
Personal_or_Business                     object
Placement_-_Day_of_Month                  int64
Placement_-_Weekday_(Mo_=_1)              int64
Placement_-_Time                         object
Confirmation_-_Day_of_Month               int64
Confirmation_-_Weekday_(Mo_=_1)           int64
Confirmation_-_Time                      object
Arrival_at_Pickup_-_Day_of_Month          int64
Arrival_at_Pickup_-_Weekday_(Mo_=_1)      int64
Arrival_at_Pickup_-_Time                 object
Pickup_-_Day_of_Month                     int64
Pickup_-_Weekday_(Mo_=_1)                 int64
Pickup_-_Time                            object
Distance_(KM)                             int64
Temperature                             float64
Precipitation_in_millimeters            float64
Pickup_Lat                              

In [44]:
df1.corr()

,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Confirmation_-_Day_of_Month,Confirmation_-_Weekday_(Mo_=_1),Arrival_at_Pickup_-_Day_of_Month,Arrival_at_Pickup_-_Weekday_(Mo_=_1),Pickup_-_Day_of_Month,Pickup_-_Weekday_(Mo_=_1),Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Time_from_Pickup_to_Arrival
Platform_Type,1.000000,0.013958,-0.058637,0.013963,-0.058612,0.013963,-0.058612,0.013963,-0.058612,0.005981,0.220127,-0.050634,-0.003386,0.019767,0.006558,0.042425,-0.003827
Placement_-_Day_of_Month,0.013958,1.000000,0.023910,0.999999,0.023890,0.999999,0.023890,0.999999,0.023890,-0.001978,0.008006,-0.223946,-0.005092,0.000761,0.003882,0.003052,-0.014710
Placement_-_Weekday_(Mo_=_1),-0.058637,0.023910,1.000000,0.023898,0.999964,0.023898,0.999964,0.023898,0.999964,0.009394,0.030172,-0.031377,0.000214,0.010483,0.002670,-0.000915,0.009693
Confirmation_-_Day_of_Month,0.013963,0.999999,0.023898,1.000000,0.023891,1.000000,0.023891,1.000000,0.023891,-0.001979,0.008007,-0.223946,-0.005087,0.000764,0.003888,0.003049,-0.014701
Confirmation_-_Weekday_(Mo_=_1),-0.058612,0.023890,0.999964,0.023891,1.000000,0.023891,1.000000,0.023891,1.000000,0.009388,0.030179,-0.031377,0.000241,0.010503,0.002706,-0.000934,0.009744
Arrival_at_Pickup_-_Day_of_Month,0.013963,0.999999,0.023898,1.000000,0.023891,1.000000,0.023891,1.000000,0.023891,-0.001979,0.008007,-0.223946,-0.005087,0.000764,0.003888,0.003049,-0.014701
Arrival_at_Pickup_-_Weekday_(Mo_=_1),-0.058612,0.023890,0.999964,0.023891,1.000000,0.023891,1.000000,0.023891,1.000000,0.009388,0.030179,-0.031377,0.000241,0.010503,0.002706,-0.000934,0.009744
Pickup_-_Day_of_Month,0.013963,0.999999,0.023898,1.000000,0.023891,1.000000,0.023891,1.000000,0.023891,-0.001979,0.008007,-0.223946,-0.005087,0.000764,0.003888,0.003049,-0.014701
Pickup_-_Weekday_(Mo_=_1),-0.058612,0.023890,0.999964,0.023891,1.000000,0.023891,1.000000,0.023891,1.000000,0.009388,0.030179,-0.031377,0.000241,0.010503,0.002706,-0.000934,0.009744
Distance_(KM),0.005981,-0.001978,0.009394,-0.001979,0.009388,-0.001979,0.009388,-0.001979,0.009388,1.000000,-0.004892,-0.039817,-0.128315,0.180356,-0.076642,0.145198,0.580608


In [45]:
df1 = df1.drop(['Placement_-_Weekday_(Mo_=_1)', 'Placement_-_Weekday_(Mo_=_1)', 'Confirmation_-_Day_of_Month', 'Confirmation_-_Weekday_(Mo_=_1)'
        , 'Arrival_at_Pickup_-_Day_of_Month','Arrival_at_Pickup_-_Weekday_(Mo_=_1)', 'Pickup_-_Day_of_Month','Pickup_-_Weekday_(Mo_=_1)',], axis = 1)

In [46]:
df1.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Pickup_-_Time,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Rider_Id,Time_from_Pickup_to_Arrival
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,4:44:10 PM,4:44:29 PM,4:53:04 PM,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192,NaN
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,12:57:35 PM,12:59:17 PM,1:20:27 PM,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868,NaN
2,Order_No_768,User_Id_2154,Bike,3,Business,27,11:08:14 AM,11:25:05 AM,11:33:20 AM,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26,NaN
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1:51:35 PM,1:53:27 PM,2:02:41 PM,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685,NaN
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,11:30:28 AM,11:34:45 AM,11:47:19 AM,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858,NaN


**NaN Value Checks**

In [47]:
df1.isnull().sum()

Order_No                            0
User_Id                             0
Vehicle_Type                        0
Platform_Type                       0
Personal_or_Business                0
Placement_-_Day_of_Month            0
Placement_-_Time                    0
Confirmation_-_Time                 0
Arrival_at_Pickup_-_Time            0
Pickup_-_Time                       0
Distance_(KM)                       0
Temperature                      5803
Precipitation_in_millimeters    27518
Pickup_Lat                          0
Pickup_Long                         0
Destination_Lat                     0
Destination_Long                    0
Rider_Id                            0
Time_from_Pickup_to_Arrival      7068
dtype: int64

In [52]:
df1['Temperature'].fillna((df1['Temperature'].median()), inplace=True)

In [53]:
df1['Precipitation_in_millimeters'].fillna((df1['Precipitation_in_millimeters'].median()), inplace=True)

In [54]:
df1['Time_from_Pickup_to_Arrival'].fillna((df1['Time_from_Pickup_to_Arrival'].median()), inplace=True)

In [55]:
df1.isnull().sum()

Order_No                        0
User_Id                         0
Vehicle_Type                    0
Platform_Type                   0
Personal_or_Business            0
Placement_-_Day_of_Month        0
Placement_-_Time                0
Confirmation_-_Time             0
Arrival_at_Pickup_-_Time        0
Pickup_-_Time                   0
Distance_(KM)                   0
Temperature                     0
Precipitation_in_millimeters    0
Pickup_Lat                      0
Pickup_Long                     0
Destination_Lat                 0
Destination_Long                0
Rider_Id                        0
Time_from_Pickup_to_Arrival     0
dtype: int64

In [56]:
df1.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Pickup_-_Time,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Rider_Id,Time_from_Pickup_to_Arrival
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,4:44:10 PM,4:44:29 PM,4:53:04 PM,5:06:47 PM,8,23.4,2.9,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192,1369.0
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,12:57:35 PM,12:59:17 PM,1:20:27 PM,1:25:37 PM,5,23.4,2.9,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868,1369.0
2,Order_No_768,User_Id_2154,Bike,3,Business,27,11:08:14 AM,11:25:05 AM,11:33:20 AM,11:57:54 AM,5,22.8,2.9,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26,1369.0
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1:51:35 PM,1:53:27 PM,2:02:41 PM,2:16:52 PM,5,24.5,2.9,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685,1369.0
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,11:30:28 AM,11:34:45 AM,11:47:19 AM,11:56:04 AM,6,24.4,2.9,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858,1369.0


**Dropping irrelevant columns**

In [57]:
df1 = df1.drop(['Order_No','User_Id','Vehicle_Type','Rider_Id'],axis=1) # test merging the rider dataset and then don't drop rider ID
df1.head()

,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Pickup_-_Time,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Time_from_Pickup_to_Arrival
0,3,Business,27,4:44:10 PM,4:44:29 PM,4:53:04 PM,5:06:47 PM,8,23.4,2.9,-1.333275,36.870815,-1.305249,36.822390,1369.0
1,3,Business,17,12:57:35 PM,12:59:17 PM,1:20:27 PM,1:25:37 PM,5,23.4,2.9,-1.272639,36.794723,-1.277007,36.823907,1369.0
2,3,Business,27,11:08:14 AM,11:25:05 AM,11:33:20 AM,11:57:54 AM,5,22.8,2.9,-1.290894,36.822971,-1.276574,36.851365,1369.0
3,3,Business,17,1:51:35 PM,1:53:27 PM,2:02:41 PM,2:16:52 PM,5,24.5,2.9,-1.290503,36.809646,-1.303382,36.790658,1369.0
4,3,Business,11,11:30:28 AM,11:34:45 AM,11:47:19 AM,11:56:04 AM,6,24.4,2.9,-1.281081,36.814423,-1.266467,36.792161,1369.0


**Encoding catergorical**

In [58]:
df1 = df1.drop(['Placement_-_Time','Confirmation_-_Time','Arrival_at_Pickup_-_Time','Pickup_-_Time'],axis=1)

In [59]:
df1.dtypes

Platform_Type                     int64
Personal_or_Business             object
Placement_-_Day_of_Month          int64
Distance_(KM)                     int64
Temperature                     float64
Precipitation_in_millimeters    float64
Pickup_Lat                      float64
Pickup_Long                     float64
Destination_Lat                 float64
Destination_Long                float64
Time_from_Pickup_to_Arrival     float64
dtype: object

In [60]:
df1 = pd.get_dummies(df1)
df1.head()

,Platform_Type,Placement_-_Day_of_Month,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Time_from_Pickup_to_Arrival,Personal_or_Business_Business,Personal_or_Business_Personal
0,3,27,8,23.4,2.9,-1.333275,36.870815,-1.305249,36.822390,1369.0,1,0
1,3,17,5,23.4,2.9,-1.272639,36.794723,-1.277007,36.823907,1369.0,1,0
2,3,27,5,22.8,2.9,-1.290894,36.822971,-1.276574,36.851365,1369.0,1,0
3,3,17,5,24.5,2.9,-1.290503,36.809646,-1.303382,36.790658,1369.0,1,0
4,3,11,6,24.4,2.9,-1.281081,36.814423,-1.266467,36.792161,1369.0,1,0


**Modelling**

In [61]:
df1.head()

,Platform_Type,Placement_-_Day_of_Month,Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Time_from_Pickup_to_Arrival,Personal_or_Business_Business,Personal_or_Business_Personal
0,3,27,8,23.4,2.9,-1.333275,36.870815,-1.305249,36.822390,1369.0,1,0
1,3,17,5,23.4,2.9,-1.272639,36.794723,-1.277007,36.823907,1369.0,1,0
2,3,27,5,22.8,2.9,-1.290894,36.822971,-1.276574,36.851365,1369.0,1,0
3,3,17,5,24.5,2.9,-1.290503,36.809646,-1.303382,36.790658,1369.0,1,0
4,3,11,6,24.4,2.9,-1.281081,36.814423,-1.266467,36.792161,1369.0,1,0


In [63]:
# train test split
y = df1[:len(train_df)][['Time_from_Pickup_to_Arrival']]
x_train = df1[:len(train_df)].drop('Time_from_Pickup_to_Arrival',axis=1)
x_test = df1[len(train_df):].drop('Time_from_Pickup_to_Arrival',axis=1)
x = df1[:len(train_df)].drop('Time_from_Pickup_to_Arrival',axis=1)

**Load Model**

In [64]:
lr = LinearRegression()
####try different models on different versions, go wild! to see results

In [65]:
# Using just the training dataset to test model accuracy 
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state=42) 

**Fitting Model**

In [66]:
lr.fit(x_train,y_train)

preds = lr.predict(x_test)

In [67]:
def rmse(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))

In [68]:
rmse(y_test, preds)

739.1373483642863

In [69]:
from sklearn.metrics import r2_score

In [70]:
r2_score(y_test, preds) 
### checking for accuracy

0.21882006608223892

# Making a submission

In [71]:
x_train = df[:len(train_df)].drop('Time_from_Pickup_to_Arrival',axis=1)
x_test = df[len(train_df):].drop('Time_from_Pickup_to_Arrival',axis=1)

In [72]:
lr.fit(x_train,y)

preds = lr.predict(x_test)

In [73]:
preds

array([[1529.36646186],
       [1486.35107376],
       [1500.40190691],
       ...,
       [1491.74756087],
       [1496.54118966],
       [1478.93308595]])

In [74]:
daf=pd.DataFrame(preds, columns=['Time from Pickup to Arrival'])
daf.head()

,Time from Pickup to Arrival
0,1529.366462
1,1486.351074
2,1500.401907
3,1499.295873
4,1490.490290


In [75]:
output = pd.DataFrame({'Order No':test_df['Order_No']})
submission = output.join(daf)        
submission.to_csv("submission.csv", index=False)

In [76]:
submission.head()

,Order No,Time from Pickup to Arrival
0,Order_No_19248,1529.366462
1,Order_No_12736,1486.351074
2,Order_No_768,1500.401907
3,Order_No_15332,1499.295873
4,Order_No_21373,1490.490290


In [ ]:
print("Sendy Submission : 792.392440722007")

Sendy Submission : 792.392440722007
